In [ ]:
from quantplay.strategy.base import QuantplayAlgorithm
from quantplay.utils.constant import TickInterval
from quantplay.service import market
import numpy as np
import pandas as pd
import talib

class RSI(QuantplayAlgorithm):
    def __init__(self):
        self.interval = "5minute"
        self.entry_time = "09:25"
        self.exit_time = "15:15"
        self.exchange_to_trade_on = "NSE"
        self.stream_symbols_by_security_type = {"EQ": market.symbols(universe="FNO_STOCKS")}
        self.strategy_type = "intraday"
        self.strategy_tag = "rsi"

        super(RSI, self).__init__()

    def get_trades(self, market_data):
        trades = market.get_trades(market_data, self.entry_time)

        trades.loc[:, 'rsi'] = talib.RSI(trades.close, timeperiod=7)
        trades = trades[trades.rsi > 70]

        trades.loc[:, 'tradingsymbol'] = trades.symbol
        trades.loc[:, "transaction_type"] = "SELL"
        trades.loc[:, "stoploss"] = 0.03
        trades.loc[:, "quantity"] = (100000/trades.close).astype(int)

        return trades

In [5]:
strategy = RSI()
strategy.validate()
res, trades = strategy.backtest()